In [18]:
import numpy as np
import sys
import pickle
from os import mkdir
from os.path import join, isdir
from numpy import linalg as LA
from math import sqrt, inf
from decimal import Decimal
import time
import gym
import envs
from gym import spaces, logger
from scenario_objects import Point, Cell, User, Environment
import plotting
from my_utils import *
import agent
from Astar import *
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
'''# _________ Main training parameters:_________
'''
SHOW_EVERY = 30
LEARNING_RATE = 1.0
DISCOUNT = 0.95
EPSILON = 1.0
EPSILON_DECREMENT = 0.998
EPSILON_MIN = 0.01
EPSILON_MIN2 = 0.4

max_value_for_Rmax = 100
ITERATIONS_PER_EPISODE = 40

# _____________________________________________


# __________________ Main loadings: __________________



In [19]:
env = gym.make('UAVEnv-v0')
MAX_UAV_HEIGHT = env.max_uav_height
n_actions = env.nb_actions
actions_indeces = range(n_actions)
cs_cells = env.cs_cells
cs_cells_coords_for_UAVs = [(cell._x_coord, cell._y_coord) for cell in cs_cells] if DIMENSION_2D==True else [(cell._x_coord, cell._y_coord, cell._z_coord) for cell in cs_cells]
#cells_matrix = env.cells_matrix
action_set_min = env.action_set_min
if (UNLIMITED_BATTERY==False):
    q_table_action_set = env.q_table_action_set
    charging_set = env.charging_set
    come_home_set = env.come_home_set

centroids = env.cluster_centroids
# Scale centroids according to the selected resolution:
env_centroids = [(centroid[0]/CELL_RESOLUTION_PER_COL, centroid[1]/CELL_RESOLUTION_PER_ROW) for centroid in centroids]
print(action_set_min)

[5, 6, 7, 8, 4]


C:\Users\vansh\AppData\Roaming\Python\Python39\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [20]:
states = env.observation_space.shape
actions = env.action_space.n


In [21]:
n_actions

7

In [22]:
actions

7

In [23]:
states

(10, 10)

In [24]:
length, width= env.observation_space.shape

In [27]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [28]:
import numpy as np
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Conv2D
from tensorflow.keras.optimizers import Adam

def build_model(height, width, actions):
    model = tensorflow.keras.Sequential()    
    model.add(Conv2D(24, activation='relu', input_shape=(2,height, width)))
    model.add(Conv2D(24, activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [29]:
print(env.observation_space.shape)

(10, 10)


In [30]:
model = build_model(length, width, actions)

TypeError: __init__() missing 1 required positional argument: 'kernel_size'

In [ ]:
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 2, 10, 24)         264       
                                                                 
 dense_21 (Dense)            (None, 2, 10, 24)         600       
                                                                 
 flatten_4 (Flatten)         (None, 480)               0         
                                                                 
 dense_22 (Dense)            (None, 512)               246272    
                                                                 
 dense_23 (Dense)            (None, 256)               131328    
                                                                 
 dense_24 (Dense)            (None, 7)                 1799      
                                                                 
Total params: 380,263
Trainable params: 380,263
Non-tr

In [ ]:
env.reset()

[None, None]

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=7)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
print(env.state)

None


In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam())
dqn.fit(env, nb_steps=100, visualize=False, verbose=2)

Training for 100 steps ...


ValueError: Error when checking input: expected dense_15_input to have shape (10, 10) but got array with shape (7, 2)

In [ ]:
print(model.output_shape)

In [ ]:
print(actions)

In [ ]:
env.reset =env.reset_uavs